In [55]:
import pandas as pd
from tqdm import tqdm
from nltk.stem import PorterStemmer
import text_hammer as th
from nltk.corpus import stopwords
import re
import math

df  = pd.read_csv("dataset.csv")
df2 = df[df.label == 'gpsv']

In [56]:
df2

,description,label
13014,VDSM in Red Hat Enterprise Virtualization 3 an...,gpsv
13015,Multiple unspecified vulnerabilities in Mozill...,gpsv
13016,ChakraCore and Microsoft Edge in Windows 10 Go...,gpsv
13017,Firefox before 1.0.3 and Mozilla Suite before ...,gpsv
13018,Unspecified vulnerability in the Internet Prot...,gpsv
...,...,...
44220,Integrated Management Module (IMM) 2 1.00 thro...,gpsv
44221,Admin Framework in Apple OS X before 10.10.4 d...,gpsv
44222,Insufficient policy enforcement in developer t...,gpsv
44223,Integer overflow in Adobe Reader and Acrobat 1...,gpsv


In [57]:
stop_words = set(stopwords.words("english"))
tqdm.pandas()
ps = PorterStemmer()

def remove_stopwords(text):
    try:
        return " ".join([word for word in text.split() if word not in stop_words])
    except AttributeError:
        print(text)

def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x: x.lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
    df[column] = df[column].progress_apply(lambda x: remove_stopwords(x))
    #df[column] = df[column].progress_apply(lambda x: th.remove_common_words(x, ['allow']))#, 'user', 'allow', 'service', 'version', 'file', 'attacker', 'arbitrary', 'cause', 'execute', 'denial']))
    df[column] = df[column].progress_apply(lambda x: ps.stem(x))
    #df[column] = df[column].progress_apply(lambda x: lemmatizing(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    
    df[column] = df[column].progress_apply(lambda x: th.make_base(x)) #ran -> run,
    #df[column] = df[column].progress_apply(lambda x: re.sub(r'[^\w\s//-]', '', x))
    return(df)

In [58]:
es_prep = text_preprocessing(df2, "description")
#gps_prep = text_preprocessing(gps, "description")

100%|██████████| 31211/31211 [00:00<00:00, 322379.75it/s]
/tmp/ipykernel_7359/4124639927.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].progress_apply(lambda x: x.lower())
100%|██████████| 31211/31211 [01:56<00:00, 267.35it/s]
/tmp/ipykernel_7359/4124639927.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
100%|██████████| 31211/31211 [00:00<00:00, 87905.32it/s] 
/tmp/ipykernel_7

In [38]:
import nltk
### Calculating the word frequency by using nltk
words_list = []
for sentence in es.description:
    words_list.extend(nltk.word_tokenize(sentence))
es_freq_dist = nltk.FreqDist(words_list)
es_freq_dist.most_common(100)
#print(freq_dist.keys())    
words_list = []
for sentence in es.description:
    words_list.extend(nltk.word_tokenize(sentence))
gps_freq_dist = nltk.FreqDist(words_list)
gps_freq_dist.most_common(100)

[('attacker', 8913),
 ('version', 8848),
 ('allow', 8150),
 ('android', 7480),
 ('vulnerability', 6208),
 ('application', 5910),
 ('prior', 5872),
 ('issue', 5494),
 ('via', 5195),
 ('device', 5022),
 ('craft', 4193),
 ('could', 4097),
 ('server', 3710),
 ('service', 3541),
 ('remote', 3514),
 ('affect', 3507),
 ('information', 3258),
 ('certificate', 3240),
 ('sd', 3142),
 ('cause', 3126),
 ('privilege', 3035),
 ('user', 2999),
 ('aka', 2673),
 ('arbitrary', 2612),
 ('denial', 2580),
 ('code', 2523),
 ('cisco', 2465),
 ('series', 2446),
 ('d', 2423),
 ('i', 2420),
 ('obtain', 2391),
 ('sensitive', 2314),
 ('io', 2258),
 ('execute', 2183),
 ('may', 2157),
 ('use', 2145),
 ('network', 2105),
 ('access', 2043),
 ('product', 1996),
 ('local', 1990),
 ('system', 1960),
 ('lead', 1854),
 ('need', 1807),
 ('memory', 1796),
 ('packet', 1793),
 ('due', 1773),
 ('ios', 1743),
 ('exploit', 1738),
 ('execution', 1713),
 ('spoof', 1708),
 ('maninthemiddle', 1657),
 ('ssl', 1637),
 ('os', 1631),
 (

In [43]:
es_freq_dist.most_common(10)

[('attacker', 8913),
 ('version', 8848),
 ('allow', 8150),
 ('android', 7480),
 ('vulnerability', 6208),
 ('application', 5910),
 ('prior', 5872),
 ('issue', 5494),
 ('via', 5195),
 ('device', 5022)]

In [21]:
# Term frequency
def tf(word_freq):
    freq_sum  = 0
    words_tf = {}
    for word in word_freq.keys():
        freq_sum += word_freq[word]
    
    for word in word_freq.keys():
        words_tf[word] = word_freq[word] / freq_sum
    return words_tf

In [22]:
es_tf = tf(es_freq_dist)
gps_tf = tf(gps_freq_dist)

In [32]:
def word_exist(word, text):
    text = text.lower()
    return True if re.search(word, text) else False

succes!


In [ ]:
# Inverse-Document frequency
def idf(words_dic, n, df, category):
    words = words_dic.keys()
    occurance_freq = {key: 0 for key in words}
    total_num_text = {key: 0 for key in words}
    words_idf = {}
    for word in words:
        for text in category.description:
            if word_exist(word, text):
                occurance_freq[word] += 1
    for word in words:
        for text in df.description:
            if word_exist(word, text):
                total_num_text[word] += 1
    for word in words:
        words_idf[word] = math.log((occurance_freq[word]*n)/total_num_text[word], 10)

    return words_idf
    

   

In [ ]:
n_tf_idf = {}

